## Ensemble

### 공통코드

In [8]:
# 파이썬 ≥3.5 필수
import sys
assert sys.version_info >= (3, 5)
# sklearn ≥0.20 필수
import sklearn
assert sklearn.__version__ >= "0.20"
# 공통 모듈 임포트
import numpy as np
import pandas as pd
import seaborn as sns
import os

# 노트북 실행 결과를 동일하게 유지하기 위해
np.random.seed(42)
# 깔끔한 그래프 출력을 위해
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
# 그림을 저장할 위치
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "regression"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

import platform
from matplotlib import font_manager, rc
#매킨토시의 경우
if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
#윈도우의 경우
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
mpl.rcParams['axes.unicode_minus'] = False
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("그림 저장:", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

## 투표기반 분류기의 성능 확인

In [9]:
# 데이터 생성
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
print(X[0:5])
print(y[0:5])

[[ 0.83103915 -0.25874875]
 [ 1.18506381  0.92038714]
 [ 1.16402213 -0.45552558]
 [-0.0236556   1.08628844]
 [ 0.48050273  1.50942444]]
[1 0 1 0 0]


In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression(solver="lbfgs", random_state=42)
rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)
svm_clf = SVC(gamma="scale", random_state=42)

voting_clf = VotingClassifier(estimators=[('lr', log_clf), ('rf', rnd_clf), 
                                          ('svc', svm_clf)], voting='hard')

voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(random_state=42)),
                             ('rf', RandomForestClassifier(random_state=42)),
                             ('svc', SVC(random_state=42))])

In [12]:
# 각 모델의 정확도 확인
from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__,'정확도 :', accuracy_score(y_test, y_pred))

# 개별 분류기로 가장 높게 나온 RandomForest 분류기보다 투표 기반의 VotingClassifier 가 더 정확함

LogisticRegression 정확도 : 0.864
RandomForestClassifier 정확도 : 0.896
SVC 정확도 : 0.896
VotingClassifier 정확도 : 0.912


In [14]:
log_clf = LogisticRegression(solver="lbfgs", random_state=42)
rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)
svm_clf = SVC(gamma="scale", probability=True, random_state=42)

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)], 
    voting='soft')
    
voting_clf.fit(X_train, y_train)

from sklearn.metrics import accuracy_score
for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__,'정확도 :',  accuracy_score(y_test, y_pred))

LogisticRegression 정확도 : 0.864
RandomForestClassifier 정확도 : 0.896
SVC 정확도 : 0.896
VotingClassifier 정확도 : 0.92


### Bagging을 이용한 앙상블

In [16]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=100, bootstrap=True, random_state=42)

bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

print(accuracy_score(y_test, y_pred))

0.904


In [17]:
tree_clf = DecisionTreeClassifier(random_state=42)
tree_clf.fit(X_train, y_train)
y_pred_tree = tree_clf.predict(X_test)
print(accuracy_score(y_test, y_pred_tree))

0.856


In [ ]:
#oob 평가
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=100, bootstrap=True, random_state=42, oob_score=True
)
bag_clf.fit(X_train, y_train)
print(bag_clf.oob_score_)

In [ ]:
#배깅
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(max_features="sqrt", max_leaf_nodes=16),
    n_estimators=500,
    random_state=42)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

#랜덤 포레스트
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16,
                                random_state=42)
rnd_clf.fit(X_train, y_train)
y_pred_rf = rnd_clf.predict(X_test)

print(np.sum(y_pred == y_pred_rf) / len(y_pred))

#결과는 1.0 : bag_clf 와 rnd_clf 가 예측한 결과가 정확히 일치

In [ ]:
df = sns.load_dataset('titanic')
df

In [ ]:
# 결측치 처리
rdf = df.drop(['deck', 'embark_town'], axis=1)
rdf = rdf.dropna(subset=['age'], how='any', axis=0)
most_freq=rdf['embarked'].value_counts(dropna=True).idxmax()
rdf['embarked'].fillna(most_freq, inplace=True)
rdf.info()

In [ ]:
#분석에 사용할 데이터 골라내기
ndf = rdf[['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'embarked']]

#순서가 없는 범주형 데이터의 원 핫 인코딩
onehot_sex = pd.get_dummies(ndf['sex'])
ndf = pd.concat([ndf, onehot_sex], axis=1)

onehot_embarked = pd.get_dummies(ndf['embarked'], prefix='town')
ndf = pd.concat([ndf, onehot_embarked], axis=1)

ndf.drop(['sex', 'embarked'],axis=1, inplace=True)

print(ndf.head())

In [ ]:
#피처 와 타겟을 분리
X = ndf[['pclass', 'age', 'sibsp', 'parch', 'female', 'male', 'town_C',
        'town_Q', 'town_S']]
y = ndf['survived']

#정규화
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)
print(X[0:5])

In [ ]:
#훈련 데이터 와 테스트 데이터 분할

#데이터가 랜덤하게 배치되어 있다면 분할을 할 때 순서대로 앞에서 일정 부분은
#훈련 데이터로 사용하고 일정 부분은 테스트 데이터로 할당하면 됨

#타겟의 비율을 고려 
#타겟의 비율이 고르지 않다면 여러가지 고려
#층화 추출이나 oversampling 이나 undersampling
#이상치 탐지에서 이 부분이 항상 고려 대상

#훈련 데이터의 비율 - 일반적으로 0.7 이나 0.8을 많이 사용
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)


In [ ]:
#불순도 지표는 entropy - 기본은 gini 계수
#트리의 개수는 25개
#코어는 최대한 사용
forest = RandomForestClassifier(criterion='entropy', 
                                n_estimators=25, random_state=42, n_jobs=-1)

forest.fit(X_train, y_train)

In [ ]:
y_hat = forest.predict(X_test)

print(y_hat[0:10])
print(y_test.values[0:10])

from sklearn import metrics
#오차 행렬 확인
tree_matrix = metrics.confusion_matrix(y_test, y_hat)
print(tree_matrix)
#정확도 계산
print((tree_matrix[0, 0] + tree_matrix[1,1]) / np.sum(tree_matrix))

tree_report = metrics.classification_report(y_test, y_hat)
print(tree_report)

In [ ]:
n_features = X.data.shape[1]
plt.barh(np.arange(n_features), forest.feature_importances_, align='center')
plt.yticks(np.arange(n_features), 
           ['pclass', 'age', 'sibsp', 'parch', 'female', 'male', 'town_C', 'town_Q',
           'town_S'])
plt.xlabel('특성 중요도')
plt.ylabel('특성')
plt.ylim(-1, n_features)
plt.show()


In [ ]:
## AdaBoost
from sklearn.ensemble import AdaBoostClassifier
#algorithm 은 확률을 계산할 수 있으면 SAMME.R 이고 없으면 SAMME
#learning_rate 는 학습률
#학습률이 너무 크면 최적화 되지 않을 가능성이 높아지고 너무 작으면 훈련 속도가 느려지고
#overfitting 될 가능성이 발생
ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1),
    n_estimators=200,
    algorithm="SAMME.R", 
    learning_rate=0.5,
    random_state=42
)

ada_clf.fit(X_train, y_train)

y_hat = ada_clf.predict(X_test)

print(y_hat[0:10])
print(y_test.values[0:10])
#오차 행렬 확인
ada_matrix = metrics.confusion_matrix(y_test, y_hat)
print(ada_matrix)
#정확도 계산
print((ada_matrix[0, 0] + ada_matrix[1,1]) / np.sum(ada_matrix))

ada_report = metrics.classification_report(y_test, y_hat)
print(ada_report)

In [ ]:
%%time
#주피터 노트북에서 시간 측정할 때 사용
## Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier
gbm_clf = GradientBoostingClassifier(random_state=42)

gbm_clf.fit(X_train, y_train)

y_hat = gbm_clf.predict(X_test)

print(y_hat[0:10])
print(y_test.values[0:10])

In [ ]:
%%time
#하이퍼 파라미터 튜닝 - 시간이 있을 때는 여러 값을 설정
from sklearn.model_selection import GridSearchCV

#실제로는 4가지
params = {
    'n_estimators':[100, 500],
    'learning_rate': [0.05, 0.1]
}

grid_cv = GridSearchCV(gbm_clf, param_grid=params, cv=2, verbose=1)
grid_cv.fit(X_train, y_train)
print("최적의 파라미터:", grid_cv.best_params_)
print("최적의 정확도:", grid_cv.best_score_)

#튜닝의 결과로 예측
gbm_pred = grid_cv.best_estimator_.predict(X_test)
gbm_report = metrics.classification_report(y_test, gbm_pred)
print(gbm_report)